## Q3 ~ Amy

In [8]:

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import json
import pandas as pd

import plotly.graph_objects as go
from datetime import datetime, timedelta
import time
import numpy as np

import scipy
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import matplotlib as plt
import seaborn as sns

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# store login data in login.py
%run login.py

loginquery = f"""
mutation {{
  logIn(
      email:\"{login}\",
      password:\"{pwd}\") {{
    jwt {{
      token
      exp
    }}
  }}
}}
"""
import requests
url = 'https://api.numina.co/graphql'

mylogin = requests.post(url, json={'query': loginquery})
token = mylogin.json()['data']['logIn']['jwt']['token']
expdate = mylogin.json()['data']['logIn']['jwt']['exp']

In [9]:
query1 = """
query {
  feedCountMetrics(
    serialnos:["SWLSANDBOX1"],
    startTime:"2018-01-01T00:00:00",
    endTime:"2019-12-31T00:00:00",
    objClasses:["pedestrian"],
    timezone:"America/New_York",
    interval:"24h") {
    edges {
      node {
        serialno
        result
        objClass
        time
      }
    }
  }
}
"""
query2 = """
query {
  feedCountMetrics(
    serialnos:["SWLSANDBOX2"],
    startTime:"2018-01-01T00:00:00",
    endTime:"2019-12-31T00:00:00",
    objClasses:["pedestrian"],
    timezone:"America/New_York",
    interval:"24h") {
    edges {
      node {
        serialno
        result
        objClass
        time
      }
    }
  }
}
"""
query3 = """
query {
  feedCountMetrics(
    serialnos:["SWLSANDBOX3"],
    startTime:"2018-01-01T00:00:00",
    endTime:"2019-12-31T00:00:00",
    objClasses:["pedestrian"],
    timezone:"America/New_York",
    interval:"24h") {
    edges {
      node {
        serialno
        result
        objClass
        time
      }
    }
  }
}
"""




In [10]:
# count 500 hrs when there is a pedestrian passing by
hourquery1 = """
query {
  feedCountMetrics(
    serialnos:["SWLSANDBOX1"],
    startTime:"2018-01-01T00:00:00",
    endTime:"2019-12-31T00:00:00",
    objClasses:["pedestrian"],
    timezone:"America/New_York",
    interval:"1h") {
    edges {
      node {
        serialno
        result
        objClass
        time
      }
    }
  }
}
"""
hourquery2 = """
query {
  feedCountMetrics(
    serialnos:["SWLSANDBOX2"],
    startTime:"2018-01-01T00:00:00",
    endTime:"2019-12-31T00:00:00",
    objClasses:["pedestrian"],
    timezone:"America/New_York",
    interval:"1h") {
    edges {
      node {
        serialno
        result
        objClass
        time
      }
    }
  }
}
"""
hourquery2 = """
query {
  feedCountMetrics(
    serialnos:["SWLSANDBOX3"],
    startTime:"2018-01-01T00:00:00",
    endTime:"2019-12-31T00:00:00",
    objClasses:["pedestrian"],
    timezone:"America/New_York",
    interval:"1h") {
    edges {
      node {
        serialno
        result
        objClass
        time
      }
    }
  }
}
"""



devices = {'Streetscape': ('SWLSANDBOX1', 'img/Streetscape.png'), 'Outside': ('SWLSANDBOX3', 'img/Outside.png'),
           'UnderRaincoat': ('SWLSANDBOX2', 'img/UnderRaincoat.png')}

In [11]:
def create_df(sensor):
    df = pd.DataFrame(columns = ['Date', 'Number of Pedestrians', "Day of Week"])
    if sensor == 'Streetscape':
        peds = requests.post(url, json={'query': query1}, headers = {'Authorization':token})
       
    if sensor == 'Outside':
        peds = requests.post(url, json={'query': query3}, headers = {'Authorization':token})
       
    if sensor == 'UnderRaincoat':
        peds = requests.post(url, json={'query': query2}, headers = {'Authorization':token})
        
    p = peds.json()['data']['feedCountMetrics']['edges']  
    
    for node in p:
        date = datetime.strptime(node['node']['time'][:10], '%Y-%m-%d')
        df = df.append({'Date': date, 'Number of Pedestrians': node['node']['result'], "Day of Week" : date.strftime('%A')}, ignore_index=True)
    
    df.set_index('Date', inplace=True)
    df['Number of Pedestrians'] = df['Number of Pedestrians'].astype('int32')
    return df


In [12]:
def remove_outliers(df):
    testing_df = df.drop(df[df['Number of Pedestrians'] == 0].index)
    q1 = testing_df['Number of Pedestrians'].quantile(0.10)
    q3 = testing_df['Number of Pedestrians'].quantile(0.90)

    interquartile_range = q3 - q1

    df = df[~((df['Number of Pedestrians'] < (q1)) \
             |(df['Number of Pedestrians'] > (q3)))]
    return df


In [13]:
def create_hourly_df(sensor):
    if sensor == 'Streetscape':
        hour_peds = requests.post(url, json={'query': hourquery1}, headers = {'Authorization':token})
       
    if sensor == 'Outside':
        hour_peds = requests.post(url, json={'query': hourquery3}, headers = {'Authorization':token})
       
    if sensor == 'UnderRaincoat':
        hour_peds = requests.post(url, json={'query': hourquery1}, headers = {'Authorization':token})
        
    ph = hour_peds.json()['data']['feedCountMetrics']['edges']
    
    hourly = pd.DataFrame(columns = ['Date', 'Number of Pedestrians', "Day of Week", "Hour"])
    for node in ph:
        date = node['node']['time'][:10]
        hr = node['node']['time'][11:16]
        time = date +' '+ hr
        date = datetime.strptime(time, '%Y-%m-%d %H:%M')
        hourly = hourly.append({'Date': date, 'Number of Pedestrians': node['node']['result'], "Day of Week" : date.strftime('%A'), "Hour": hr}, ignore_index=True)
    
    hourly.set_index('Date', inplace=True)

    hourly['Number of Pedestrians'] = hourly['Number of Pedestrians'].astype('int32')
    
    return hourly


In [14]:
import matplotlib.pyplot as plt
def next_weekday(d, weekday):
    days_ahead = weekday - d.weekday()
    if days_ahead <= 0: # Target day already happened this week
        days_ahead += 7
    return d + timedelta(days_ahead)

def get_maintenance(hours, device, day, start_date, end_date):
    saved_dates = []
    if device == 'Streetscape':
        hourly = create_hourly_df('Streetscape')
    if device == 'Outside':
        hourly = create_hourly_df('Outside')
    if device == 'UnderRaincoat':
        hourly = create_hourly_df('UnderRaincoat')
        
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    
    plot_df = hourly.loc[(hourly.index >= start_date) & (hourly.index <= end_date)].copy()
    
    dates = []
    count = 0
    for index, row in plot_df.iterrows():
        if row['Number of Pedestrians'] > 0 & row['Number of Pedestrians'] < hours:
            count += 1
        if row['Number of Pedestrians'] >= hours/2:
            count += row['Number of Pedestrians']
        if count >= hours:
            if day == "Any":
                d = index + timedelta(days=1)
            else:
                d = next_weekday(index, time.strptime(day, "%A").tm_wday)
            d = d.replace(hour=16, minute=0, second=0)
            saved_dates.append(d)
            count = 0
    dates = pd.Series(saved_dates) 
    dates = dates.rename("Maintenance Dates")
    dates.drop_duplicates(keep='first', inplace=True)
    dates.to_list()
    
    if device == 'Streetscape':
        df = create_df('Streetscape')
    
    if device == 'Outside':
        df = create_df('Outside')
        
    if device == 'UnderRaincoat':
        df = create_df('UnderRaincoat')
        

    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    
    plot_df = df.loc[(df.index >= start_date) & (df.index <= end_date)].copy()
    fig = go.Figure()

    # Add scatter trace for line
    fig.add_trace(go.Scatter(
        x=plot_df.index,
        y=plot_df['Number of Pedestrians'],
        mode="lines",
        name="Pedestrian Counts"
    ))
    shapes = list()
    for date in dates:
            shapes.append(
                dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=date,
                y0=0,
                x1= date + timedelta(hours=9),
                y1=1,
                fillcolor="Red",
                opacity=0.5,
                layer="above",
                line_width=0,
            ))   
    
    fig.update_layout(
        shapes = shapes,
        title=f'Maintenance schedule between {start_date.date()} and {end_date.date()}.',
        xaxis_title="Date",
        yaxis_title="Number of Pedestrians",
    )
    # plotting the large maintenance schedule
    iplot(fig)
    dates = dates.to_frame()
    
    dates=dates.reset_index(drop = True)
    dates.to_csv('maintenance.txt', sep=' ', index=False)
    
    fig, ax = plt.subplots(1, 2, figsize=(15, 7))
    
    # plotting the weekday barplot
    d = remove_outliers(df)
    plot_df = d.loc[(d.index >= start_date) & (d.index <= end_date)].copy()
    order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    ax2 = sns.barplot(x="Day of Week", y='Number of Pedestrians', data= plot_df.groupby(['Day of Week'])['Number of Pedestrians'].mean().reset_index(), order = order, palette = 'muted', ax = ax[0])
    ax2.set_title(f'Mean Counts for Pedestrians by Weekday Without Outliers')
    
    # plotting the hourly barplot
    plot_df = hourly.loc[(hourly.index >= start_date) & (hourly.index <= end_date)].copy()
    
    plot_df["Hour"] = plot_df["Hour"].astype(str)
    plot_df["Hour"] = plot_df["Hour"].str.slice(0, 2, 1) 
    hour_df = pd.DataFrame()
    hour_df['Number of Pedestrians'] = plot_df.groupby('Hour')['Number of Pedestrians'].mean()
    ax = sns.barplot(x = hour_df.index, y='Number of Pedestrians', data= hour_df, color="skyblue",  ax = ax[1])
    
    for ind, label in enumerate(ax.get_xticklabels()):
        if ind % 5 == 0:  
            label.set_visible(True)
        else:
            label.set_visible(False)
    ax.set_title(f'Mean Counts for Pedestrians by Hour')

# st_maintenance = get_maintenance(street_hourly, 500)
a = interact_manual(get_maintenance, hours = (0,2000,10),
             start_date=widgets.DatePicker(value=pd.to_datetime('2019-01-01')),
             end_date=widgets.DatePicker(value=pd.to_datetime('2020-01-01')),
            device = widgets.Dropdown(options=list(devices.keys())),
            day = widgets.Dropdown(options=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "Any"], value = "Saturday"))


interactive(children=(IntSlider(value=1000, description='hours', max=2000, step=10), Dropdown(description='dev…